In [1]:
import numpy as np
import tensorflow as tf
from keras.objectives import mean_squared_error
import keras.backend as K
import keras
import csv,os, glob
import random
import pickle
from tensorflow.contrib import rnn

Using TensorFlow backend.


In [2]:
XY=np.load("/gpfs/slac/staas/fs1/g/supercdms/tf/northwestern/users/franinsu/testRun3/XY60k.npz")
X=XY["X"]
Y=XY["Y"]
Yrange = np.array([2048,20])# Ymax-Ymin
downsample_factor = 1
N=len(X)
X=X.reshape(-1,2048,2)
X = np.array([np.mean(x.reshape(-1, downsample_factor,2), 1) for x in X])
m_train = round(.95*N)
m_dev =  round(10/95*m_train)
idx = np.random.permutation(N)
X_train, X_test = X[idx[:m_train]], X[idx[m_train:]]
Y_train, Y_test = Y[idx[:m_train]], Y[idx[m_train:]]
#dev is subset of train but isn't actually used in training
X_dev, Y_dev  = X_train[-m_dev:], Y_train[-m_dev:]
print("Size train:",X_train.shape[0]-X_dev.shape[0])
print("Size dev:",X_dev.shape[0])
print("Size test:", X_test.shape[0])
print("X_train shape:", X_train.shape)

Size train: 33539
Size dev: 3946
Size test: 1973
X_train shape: (37485, 2048, 2)


In [3]:
filter_size = 8
conv_stride = 8
input_channels = X_train.shape[2]
output_filters = 32
conv_weights = tf.Variable(tf.random_normal([filter_size, input_channels, output_filters]))
max_pool_size = 6
max_pool_stride = 6
conv_bias = tf.Variable(tf.random_normal([output_filters]))
#define constants
#unrolled through 28 time steps
time_steps=X_train.shape[1] # TO EDIT
#hidden LSTM units
num_units=256
n_input=X_train.shape[2] # TO EDIT
#learning rate for adam
learning_rate=0.001
#size of batch
# batch_size=128
n_out = 1
#weights and biases of appropriate shape to accomplish above task
seed = 35
initializer = tf.contrib.layers.xavier_initializer(seed=seed)
#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_out])
device_name = "/device:GPU:0"

In [4]:
with tf.device("/device:GPU:0"):
    input = x
    conv = tf.nn.conv1d(input,conv_weights,stride = conv_stride,padding='VALID') + conv_bias
    max_pool = tf.nn.max_pool([conv],
       ksize=[1, 1, max_pool_size, 1],
       strides=[1,1,max_pool_stride,1],
       padding='VALID')[0]
    unstacked = tf.unstack(max_pool,axis=1)
    num_units = int(max_pool.shape[1])
    out_weights=tf.Variable(initializer([num_units,16]))
    out_weights2=tf.Variable(initializer([16,n_out]))
    out_bias=tf.Variable(initializer([16]))
    out_bias2=tf.Variable(initializer([n_out]))
    print('num_units:',num_units)
    lstm_layer1 = tf.nn.rnn_cell.LSTMCell(num_units,forget_bias=1,name='basic_lstm_cell1')
    lstm_layer2 = tf.nn.rnn_cell.LSTMCell(num_units,forget_bias=1,name='basic_lstm_cell2')
    outputs1,_=rnn.static_rnn(lstm_layer1,unstacked,dtype="float32")
    outputs2,_=rnn.static_rnn(lstm_layer2,outputs1,dtype="float32")
    preprediction=tf.matmul(outputs2[-1],out_weights)+out_bias
    prediction=tf.matmul(preprediction,out_weights2)+out_bias2
    #loss_function
    loss=tf.losses.mean_squared_error(predictions=prediction,labels=y)
    #optimization
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
def mae(x_e,y_e):
    print("MAE:",np.mean(np.abs(sess.run(prediction,feed_dict={x:x_e,y:y_e})-y_e)))
#initialize variables

num_units: 42


In [ ]:
init=tf.global_variables_initializer()
m_training = m_train-m_dev
batch_size = 10
n_iters = round(m_training/batch_size - 0.5)
EPOCHS = 400
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True,log_device_placement=True)) as sess:
    sess.run(init)
    for j in range(EPOCHS):
        iter=0
        while iter<n_iters:
#            if iter % 5 ==0: print('.',end="")
#            if iter % 100==0: print("")
            batch_x = X_train[iter*batch_size:(iter+1)*batch_size]
            batch_y = Y_train[iter*batch_size:(iter+1)*batch_size,0:1]
            batch_x=batch_x.reshape((batch_size,time_steps,n_input))
            sess.run(opt, feed_dict={x: batch_x, y: batch_y})
            iter=iter+1
        if j % 1 ==0:
            #los=sess.run(loss,feed_dict={x:X_train[:-m_dev],y:Y_train[:-m_dev,0:1]})
            mae(X_train[:-m_dev],Y_train[:-m_dev,0:1])
            mae(X_train[-m_dev:],Y_train[-m_dev:,0:1])
            #mae=sess.run(score,feed_dict={x:X_train[:-m_dev],y:Y_train[:-m_dev,0:1],prob:0})
            print("For Epoch ", j)
            #print("MAE: ",mae)
            #print("Loss ",los)
            print("__________________")

    mae(X_train[:-m_dev],Y_train[:-m_dev,0:1])
    mae(X_dev,Y_dev[:,0:1])